In this section, we are experimenting with OSRM and different routing techniques to create efficient routing mechanism. 

The most common use of OSRM is to read location coordinate from a CSV file such as locations.csv and create routing between these locations. Please see a sample of location.csv in Ethiopia:

name,region,country,latitude,longitude,location_type,conflict_date,population
Addis Ababa,Addis Ababa,Ethiopia,8.978098143949728,38.75857450155794,town,0,0
Mekele,Mekele,Ethiopia,13.495486756898567,39.46589089154361,town,0,0
Dire Dawa,Dire Dawa,Ethiopia,9.602835645348511,41.85443640082678,town,0,0
Adama,Adama,Ethiopia,8.526471189559459,39.25963003366467,town,0,0

Install packages that will be used. 

In [ ]:
import sys
!{sys.executable} -m pip install pandas folium polyline tsp_solver

In [ ]:
import os
import pandas as pd
import requests
import folium
import polyline

# Set the file name of the locations.csv file
locations_csv = 'ukraine-locations.csv'

# Set the path to the directory where route.geojson is stored
locations_csv_path = '.'

# Check if the CSV file exists
locations_csv_file = os.path.join(locations_csv_path, locations_csv)
if not os.path.exists(locations_csv_file):
    print(f"Error: {locations_csv} file not found.")
    exit(1)

# Read the locations from CSV
df = pd.read_csv(locations_csv_file)

# Create a map centered around the first location
start_lat = df.loc[0, 'latitude']
start_lon = df.loc[0, 'longitude']
m = folium.Map(location=[start_lat, start_lon], zoom_start=6)

# Create an empty list to store route instructions
route_instructions = []

# Iterate through each pair of locations in the DataFrame
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        # Extract location details
        name1 = df.loc[i, 'name']
        lat1 = df.loc[i, 'latitude']
        lon1 = df.loc[i, 'longitude']
        name2 = df.loc[j, 'name']
        lat2 = df.loc[j, 'latitude']
        lon2 = df.loc[j, 'longitude']

        # Prepare API request URL
        osrm_endpoint = 'http://localhost:5000/route/v1/driving'
        request_url = f"{osrm_endpoint}/{lon1},{lat1};{lon2},{lat2}?steps=true&geometries=polyline"

        try:
            # Send request to OSRM
            response = requests.get(request_url)
            response.raise_for_status()

            # Extract route data from the response
            route_data = response.json()

            # Check if response contains a route
            if 'routes' in route_data and len(route_data['routes']) > 0:
                # Extract route steps
                steps = route_data['routes'][0]['legs'][0]['steps']
                distance_sum = 0
                for step in steps:
                    # Extract step instructions
                    instruction = step['name']

                    # Extract step distance and add to the distance sum
                    distance = step['distance']
                    distance_sum += distance

                    # Append step information to the route_instructions list
                    route_instructions.append((instruction, distance_sum))

                # Extract route geometry
                geometry = route_data['routes'][0]['geometry']

                # Decode polyline string to get coordinates
                coordinates = polyline.decode(geometry)

                # Create a Folium PolyLine object
                polyline_obj = folium.PolyLine(
                    locations=coordinates,
                    color='blue',
                    weight=2,
                    opacity=0.9
                )

                # Add the PolyLine to the map
                polyline_obj.add_to(m)

        except requests.exceptions.RequestException as e:
            print(f"Error occurred during OSRM API request: {e}")
            continue

        # Add markers for the start and end locations
        folium.CircleMarker(
            location=[lat1, lon1],
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name1
        ).add_to(m)

        folium.CircleMarker(
            location=[lat2, lon2],
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name2
        ).add_to(m)

# Save the map as an HTML file
map_file = os.path.join('images', 'ukraine-route-map.html')
m.save(map_file)
print(f"Map saved as {map_file}")

# Display the map
m


In [ ]:
import os
import pandas as pd
import requests
import folium
import polyline

# Set the file name of the locations.csv file
locations_csv = 'ukraine-locations.csv'

# Set the path to the directory where route.geojson is stored
locations_csv_path = '.'

# Check if the CSV file exists
locations_csv_file = os.path.join(locations_csv_path, locations_csv)
if not os.path.exists(locations_csv_file):
    print(f"Error: {locations_csv} file not found.")
    exit(1)

# Read the locations from CSV
df = pd.read_csv(locations_csv_file)

# Create a map centered around the first location
start_lat = df.loc[0, 'latitude']
start_lon = df.loc[0, 'longitude']
m = folium.Map(location=[start_lat, start_lon], zoom_start=6)

# Create an empty set to store processed route pairs
processed_pairs = set()

# Iterate through each pair of locations in the DataFrame
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        # Extract location details
        name1 = df.loc[i, 'name']
        lat1 = df.loc[i, 'latitude']
        lon1 = df.loc[i, 'longitude']
        name2 = df.loc[j, 'name']
        lat2 = df.loc[j, 'latitude']
        lon2 = df.loc[j, 'longitude']

        # Check if the reverse pair already exists in the processed pairs set
        if (name2, name1) in processed_pairs:
            continue

        # Add the pair to the processed pairs set
        processed_pairs.add((name1, name2))

        # Prepare API request URL
        osrm_endpoint = 'http://localhost:5000/route/v1/driving'
        request_url = f"{osrm_endpoint}/{lon1},{lat1};{lon2},{lat2}?steps=true&geometries=polyline"

        try:
            # Send request to OSRM
            response = requests.get(request_url)
            response.raise_for_status()

            # Extract route data from the response
            route_data = response.json()

            # Check if response contains a route
            if 'routes' in route_data and len(route_data['routes']) > 0:
                # Extract route steps
                steps = route_data['routes'][0]['legs'][0]['steps']
                distance_sum = 0
                for step in steps:
                    # Extract step instructions
                    instruction = step['name']

                    # Extract step distance and add to the distance sum
                    distance = step['distance']
                    distance_sum += distance

                # Append step information to the route_instructions list
                route_instructions.append((instruction, distance_sum))

                # Extract route geometry
                geometry = route_data['routes'][0]['geometry']

                # Decode polyline string to get coordinates
                coordinates = polyline.decode(geometry)

                # Create a Folium PolyLine object
                polyline_obj = folium.PolyLine(
                    locations=coordinates,
                    color='blue',
                    weight=2,
                    opacity=0.9
                )

                # Add the PolyLine to the map
                polyline_obj.add_to(m)

        except requests.exceptions.RequestException as e:
            print(f"Error occurred during OSRM API request: {e}")
            continue

        # Add markers for the start and end locations
        folium.CircleMarker(
            location=[lat1, lon1],
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name1
        ).add_to(m)

        folium.CircleMarker(
            location=[lat2, lon2],
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name2
        ).add_to(m)

# Save the map as an HTML file
map_file = os.path.join('images', 'ukraine-route-map.html')
m.save(map_file)
print(f"Map saved as {map_file}")

# Display the map
m

In [ ]:
import os
import pandas as pd
import requests
import folium
import polyline

# Set the file name of the locations.csv file
locations_csv = 'ukraine-locations.csv'

# Set the path to the directory where route.geojson is stored
locations_csv_path = '.'

# Check if the CSV file exists
locations_csv_file = os.path.join(locations_csv_path, locations_csv)
if not os.path.exists(locations_csv_file):
    print(f"Error: {locations_csv} file not found.")
    exit(1)

# Read the locations from CSV
df = pd.read_csv(locations_csv_file)

# Create a map centered around the first location
start_lat = df.loc[0, 'latitude']
start_lon = df.loc[0, 'longitude']
m = folium.Map(location=[start_lat, start_lon], zoom_start=6)

# Create an empty set to store processed route pairs
processed_pairs = set()

# Iterate through each pair of locations in the DataFrame
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        # Extract location details
        name1 = df.loc[i, 'name']
        lat1 = df.loc[i, 'latitude']
        lon1 = df.loc[i, 'longitude']
        name2 = df.loc[j, 'name']
        lat2 = df.loc[j, 'latitude']
        lon2 = df.loc[j, 'longitude']

        # Create frozensets for the pair of locations
        location_pair = frozenset([name1, name2])

        # Check if the frozenset already exists in the processed pairs set
        if location_pair in processed_pairs:
            continue

        # Add the frozenset to the processed pairs set
        processed_pairs.add(location_pair)

        # Prepare API request URL
        osrm_endpoint = 'http://localhost:5000/route/v1/driving'
        request_url = f"{osrm_endpoint}/{lon1},{lat1};{lon2},{lat2}?steps=true&geometries=polyline"

        try:
            # Send request to OSRM
            response = requests.get(request_url)
            response.raise_for_status()

            # Extract route data from the response
            route_data = response.json()

            # Check if response contains a route
            if 'routes' in route_data and len(route_data['routes']) > 0:
                # Extract route steps
                steps = route_data['routes'][0]['legs'][0]['steps']
                distance_sum = 0
                for step in steps:
                    # Extract step instructions
                    instruction = step['name']

                    # Extract step distance and add to the distance sum
                    distance = step['distance']
                    distance_sum += distance

                # Append step information to the route_instructions list
                route_instructions.append((instruction, distance_sum))

                # Extract route geometry
                geometry = route_data['routes'][0]['geometry']

                # Decode polyline string to get coordinates
                coordinates = polyline.decode(geometry)

                # Create a Folium PolyLine object
                polyline_obj = folium.PolyLine(
                    locations=coordinates,
                    color='blue',
                    weight=2,
                    opacity=0.9
                )

                # Add the PolyLine to the map
                polyline_obj.add_to(m)

        except requests.exceptions.RequestException as e:
            print(f"Error occurred during OSRM API request: {e}")
            continue

        # Add markers for the start and end locations
        folium.CircleMarker(
            location=[lat1, lon1],
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name1
        ).add_to(m)

        folium.CircleMarker(
            location=[lat2, lon2],
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name2
        ).add_to(m)

# Save the map as an HTML file
map_file = os.path.join('images', 'ukraine-route-map.html')
m.save(map_file)
print(f"Map saved as {map_file}")

# Display the map
m

In [ ]:
import os
import pandas as pd
import requests
import folium
import polyline
from sklearn.cluster import DBSCAN

# Set the file name of the locations.csv file
locations_csv = 'ukraine-locations.csv'

# Set the path to the directory where route.geojson is stored
locations_csv_path = '.'

# Check if the CSV file exists
locations_csv_file = os.path.join(locations_csv_path, locations_csv)
if not os.path.exists(locations_csv_file):
    print(f"Error: {locations_csv} file not found.")
    exit(1)

# Read the locations from CSV
df = pd.read_csv(locations_csv_file)

# Create a map centered around the first location
start_lat = df.loc[0, 'latitude']
start_lon = df.loc[0, 'longitude']
m = folium.Map(location=[start_lat, start_lon], zoom_start=6)

# Create an empty list to store route instructions
route_instructions = []

# Create a set to store visited location pairs
visited_pairs = set()

# Iterate through each pair of locations in the DataFrame
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        # Extract location details
        name1 = df.loc[i, 'name']
        lat1 = df.loc[i, 'latitude']
        lon1 = df.loc[i, 'longitude']
        name2 = df.loc[j, 'name']
        lat2 = df.loc[j, 'latitude']
        lon2 = df.loc[j, 'longitude']

        # Check if the pair has been visited before (or its reverse pair)
        if (name1, name2) in visited_pairs or (name2, name1) in visited_pairs:
            continue

        # Prepare API request URL
        osrm_endpoint = 'http://localhost:5000/route/v1/driving'
        request_url = f"{osrm_endpoint}/{lon1},{lat1};{lon2},{lat2}?steps=true&geometries=polyline"

        try:
            # Send request to OSRM
            response = requests.get(request_url)
            response.raise_for_status()

            # Extract route data from the response
            route_data = response.json()

            # Check if response contains a route
            if 'routes' in route_data and len(route_data['routes']) > 0:
                # Extract route steps
                steps = route_data['routes'][0]['legs'][0]['steps']
                distance_sum = 0
                for step in steps:
                    # Extract step instructions
                    instruction = step['name']

                    # Extract step distance and add to the distance sum
                    distance = step['distance']
                    distance_sum += distance

                # Append step information to the route_instructions list
                route_instructions.append((instruction, distance_sum))

                # Extract route geometry
                geometry = route_data['routes'][0]['geometry']

                # Decode polyline string to get coordinates
                coordinates = polyline.decode(geometry)

                # Add the coordinates to the map as a PolyLine
                folium.PolyLine(
                    locations=coordinates,
                    color='blue',
                    weight=2,
                    opacity=0.9
                ).add_to(m)

                # Add the pair to the visited set and its reverse pair
                visited_pairs.add((name1, name2))
                visited_pairs.add((name2, name1))

        except requests.exceptions.RequestException as e:
            print(f"Error occurred during OSRM API request: {e}")
            continue

        # Add markers for the start and end locations
        folium.CircleMarker(
            location=[lat1, lon1],
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name1
        ).add_to(m)

        folium.CircleMarker(
            location=[lat2, lon2],
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name2
        ).add_to(m)

# Save the map as an HTML file
map_file = os.path.join('images', 'ukraine-route-map.html')
m.save(map_file)
print(f"Map saved as {map_file}")

# Display the map
m

In [ ]:
import os
import pandas as pd
import requests
import folium
import polyline
from sklearn.cluster import DBSCAN

# Set the file name of the locations.csv file
locations_csv = 'ukraine-locations.csv'

# Set the path to the directory where route.geojson is stored
locations_csv_path = '.'

# Check if the CSV file exists
locations_csv_file = os.path.join(locations_csv_path, locations_csv)
if not os.path.exists(locations_csv_file):
    print(f"Error: {locations_csv} file not found.")
    exit(1)

# Read the locations from CSV
df = pd.read_csv(locations_csv_file)

# Create a map centered around the first location
start_lat = df.loc[0, 'latitude']
start_lon = df.loc[0, 'longitude']
m = folium.Map(location=[start_lat, start_lon], zoom_start=6)

# Create an empty list to store route instructions
route_instructions = []

# Iterate through the sequential pairs of locations in the DataFrame
for i in range(len(df)):
    # Extract location details
    name1 = df.loc[i, 'name']
    lat1 = df.loc[i, 'latitude']
    lon1 = df.loc[i, 'longitude']
    
    # Check if it is the last location
    if i == len(df) - 1:
        name2 = df.loc[0, 'name']
        lat2 = df.loc[0, 'latitude']
        lon2 = df.loc[0, 'longitude']
    else:
        name2 = df.loc[i + 1, 'name']
        lat2 = df.loc[i + 1, 'latitude']
        lon2 = df.loc[i + 1, 'longitude']

    # Prepare API request URL
    osrm_endpoint = 'http://localhost:5000/route/v1/driving'
    request_url = f"{osrm_endpoint}/{lon1},{lat1};{lon2},{lat2}?steps=true&geometries=polyline"

    try:
        # Send request to OSRM
        response = requests.get(request_url)
        response.raise_for_status()

        # Extract route data from the response
        route_data = response.json()

        # Check if response contains a route
        if 'routes' in route_data and len(route_data['routes']) > 0:
            # Extract route steps
            steps = route_data['routes'][0]['legs'][0]['steps']
            distance_sum = 0
            for step in steps:
                # Extract step instructions
                instruction = step['name']

                # Extract step distance and add to the distance sum
                distance = step['distance']
                distance_sum += distance

            # Append step information to the route_instructions list
            route_instructions.append((instruction, distance_sum))

            # Extract route geometry
            geometry = route_data['routes'][0]['geometry']

            # Decode polyline string to get coordinates
            coordinates = polyline.decode(geometry)

            # Add the coordinates to the map as a PolyLine
            folium.PolyLine(
                locations=coordinates,
                color='blue',
                weight=2,
                opacity=0.9
            ).add_to(m)

        # Add markers for the start and end locations
        folium.CircleMarker(
            location=[lat1, lon1],
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name1
        ).add_to(m)

        folium.CircleMarker(
            location=[lat2, lon2],
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name2
        ).add_to(m)

    except requests.exceptions.RequestException as e:
        print(f"Error occurred during OSRM API request: {e}")
        continue

# Save the map as an HTML file
map_file = os.path.join('images', 'ukraine-route-map.html')
m.save(map_file)
print(f"Map saved as {map_file}")

# Display the map
m

In [ ]:
import os
import pandas as pd
import requests
import folium
import polyline
from tsp_solver.greedy import solve_tsp

# Set the file name of the locations.csv file
locations_csv = 'ukraine-locations.csv'

# Set the path to the directory where route.geojson is stored
locations_csv_path = '.'

# Check if the CSV file exists
locations_csv_file = os.path.join(locations_csv_path, locations_csv)
if not os.path.exists(locations_csv_file):
    print(f"Error: {locations_csv} file not found.")
    exit(1)

# Read the locations from CSV
df = pd.read_csv(locations_csv_file)

# Create a map centered around the first location
start_lat = df.loc[0, 'latitude']
start_lon = df.loc[0, 'longitude']
m = folium.Map(location=[start_lat, start_lon], zoom_start=6)

# Create an empty list to store route instructions
route_instructions = []

# Create an empty list to store coordinates for clustering
all_coordinates = []

# Iterate through each pair of locations in the DataFrame
for i in range(len(df)):
    for j in range(len(df)):
        if i != j:
            # Extract location details
            name1 = df.loc[i, 'name']
            lat1 = df.loc[i, 'latitude']
            lon1 = df.loc[i, 'longitude']
            name2 = df.loc[j, 'name']
            lat2 = df.loc[j, 'latitude']
            lon2 = df.loc[j, 'longitude']

            # Prepare API request URL
            osrm_endpoint = 'http://localhost:5000/route/v1/driving'
            request_url = f"{osrm_endpoint}/{lon1},{lat1};{lon2},{lat2}?steps=true&geometries=polyline"

            try:
                # Send request to OSRM
                response = requests.get(request_url)
                response.raise_for_status()

                # Extract route data from the response
                route_data = response.json()

                # Check if response contains a route
                if 'routes' in route_data and len(route_data['routes']) > 0:
                    # Extract route steps
                    steps = route_data['routes'][0]['legs'][0]['steps']
                    distance_sum = 0
                    for step in steps:
                        # Extract step instructions
                        instruction = step['name']

                        # Extract step distance and add to the distance sum
                        distance = step['distance']
                        distance_sum += distance

                    # Append step information to the route_instructions list
                    route_instructions.append((instruction, distance_sum))

                    # Extract route geometry
                    geometry = route_data['routes'][0]['geometry']

                    # Decode polyline string to get coordinates
                    coordinates = polyline.decode(geometry)

                    # Add the coordinates to the list for clustering
                    all_coordinates.extend(coordinates)

            except requests.exceptions.RequestException as e:
                print(f"Error occurred during OSRM API request: {e}")
                continue

            # Add markers for the start and end locations
            folium.CircleMarker(
                location=[lat1, lon1],
                radius=6,
                color='red',
                fill=True,
                fill_color='blue',
                popup=name1
            ).add_to(m)

            folium.CircleMarker(
                location=[lat2, lon2],
                radius=6,
                color='red',
                fill=True,
                fill_color='blue',
                popup=name2
            ).add_to(m)

# Create a distance matrix for the TSP solver
n = len(all_coordinates)
dist_matrix = [[0] * n for _ in range(n)]
for i in range(n):
    for j in range(n):
        # Calculate the distance between coordinates i and j
        dist_matrix[i][j] = ((all_coordinates[i][0] - all_coordinates[j][0]) ** 2 +
                            (all_coordinates[i][1] - all_coordinates[j][1]) ** 2) ** 0.5

# Solve the TSP using the greedy solver
tsp_route = solve_tsp(dist_matrix)

# Reorder the coordinates based on the TSP route
reordered_coordinates = [all_coordinates[i] for i in tsp_route]

# Create a Folium PolyLine object for the TSP route
polyline_obj = folium.PolyLine(
    locations=reordered_coordinates,
    color='blue',
    weight=2,
    opacity=0.9
)

# Add the TSP route to the map
polyline_obj.add_to(m)

# Save the map as an HTML file
map_file = os.path.join('images', 'ukraine-route-map.html')
m.save(map_file)
print(f"Map saved as {map_file}")

# Display the map
m

In [ ]:
import os
import pandas as pd
import requests
import folium
import polyline
from sklearn.cluster import DBSCAN
from scipy.spatial import distance_matrix
from scipy.optimize import minimize

# Set the file name of the locations.csv file
locations_csv = 'ukraine-locations.csv'

# Set the path to the directory where route.geojson is stored
locations_csv_path = '.'

# Check if the CSV file exists
locations_csv_file = os.path.join(locations_csv_path, locations_csv)
if not os.path.exists(locations_csv_file):
    print(f"Error: {locations_csv} file not found.")
    exit(1)

# Read the locations from CSV
df = pd.read_csv(locations_csv_file)

# Create a map centered around the first location
start_lat = df.loc[0, 'latitude']
start_lon = df.loc[0, 'longitude']
m = folium.Map(location=[start_lat, start_lon], zoom_start=6)

# Create an empty list to store route instructions
route_instructions = []

# Create an empty list to store coordinates for clustering
all_coordinates = []

# Iterate through each pair of locations in the DataFrame
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        # Extract location details
        name1 = df.loc[i, 'name']
        lat1 = df.loc[i, 'latitude']
        lon1 = df.loc[i, 'longitude']
        name2 = df.loc[j, 'name']
        lat2 = df.loc[j, 'latitude']
        lon2 = df.loc[j, 'longitude']

        # Prepare API request URL
        osrm_endpoint = 'http://localhost:5000/route/v1/driving'
        request_url = f"{osrm_endpoint}/{lon1},{lat1};{lon2},{lat2}?steps=true&geometries=polyline"

        try:
            # Send request to OSRM
            response = requests.get(request_url)
            response.raise_for_status()

            # Extract route data from the response
            route_data = response.json()

            # Check if response contains a route
            if 'routes' in route_data and len(route_data['routes']) > 0:
                # Extract route steps
                steps = route_data['routes'][0]['legs'][0]['steps']
                distance_sum = 0
                for step in steps:
                    # Extract step instructions
                    instruction = step['name']

                    # Extract step distance and add to the distance sum
                    distance = step['distance']
                    distance_sum += distance

                # Append step information to the route_instructions list
                route_instructions.append((instruction, distance_sum))

                # Extract route geometry
                geometry = route_data['routes'][0]['geometry']

                # Decode polyline string to get coordinates
                coordinates = polyline.decode(geometry)

                # Add the coordinates to the list for clustering
                all_coordinates.extend(coordinates)

                # Add markers for the start and end locations
                folium.CircleMarker(
                    location=[lat1, lon1],
                    radius=6,
                    color='red',
                    fill=True,
                    fill_color='blue',
                    popup=name1
                ).add_to(m)

                folium.CircleMarker(
                    location=[lat2, lon2],
                    radius=6,
                    color='red',
                    fill=True,
                    fill_color='blue',
                    popup=name2
                ).add_to(m)

        except requests.exceptions.RequestException as e:
            print(f"Error occurred during OSRM API request: {e}")
            continue

# Perform DBSCAN (Density-Based Spatial Clustering of Applications with Noise) for coordinates
eps = 0.001  # distance threshold for clustering
min_samples = 2  # minimum number of points in a cluster
dbscan = DBSCAN(eps=eps, min_samples=min_samples)
cluster_labels = dbscan.fit_predict(all_coordinates)

# Create a dictionary to store polylines for each cluster
cluster_polylines = {}

# Iterate over the clusters and generate polylines for each cluster
for label in set(cluster_labels):
    if label == -1:
        # Skip noise points (not assigned to any cluster)
        continue

    # Extract coordinates for the current cluster
    cluster_coords = [coord for i, coord in enumerate(all_coordinates) if cluster_labels[i] == label]

    # Calculate the distance matrix for the cluster coordinates
    distance_matrix_cluster = distance_matrix(cluster_coords, cluster_coords)

    # Define the objective function for TSP
    def tsp_objective_function(order):
        # Cast float values to integers for indexing
        order = order.astype(int)
        return sum([distance_matrix_cluster[order[i]][order[i + 1]] for i in range(len(order) - 1)])

    # Define the constraint function for TSP
    def tsp_constraint_function(order):
        # Cast float values to integers for indexing
        order = order.astype(int)
        return sum(order) - len(order)

    # Define the initial guess for the TSP
    initial_guess = list(range(len(cluster_coords)))

    # Define the bounds for the TSP
    bounds = [(0, len(cluster_coords) - 1)] * len(cluster_coords)

    # Define the constraint for the TSP
    constraint = {'type': 'eq', 'fun': tsp_constraint_function}

    # Solve the TSP using the minimize function
    result = minimize(tsp_objective_function, initial_guess, method='SLSQP', bounds=bounds, constraints=constraint)

    # Get the optimal order of locations from the result and convert to integers
    optimal_order = result.x.astype(int)

    # Reorder the cluster coordinates based on the optimal order
    reordered_coords = [cluster_coords[i] for i in optimal_order]

    # Create a list of polyline coordinates
    polyline_coords = [(coord[0], coord[1]) for coord in reordered_coords]

    # Create a Folium PolyLine object for the current cluster
    polyline_obj = folium.PolyLine(
        locations=polyline_coords,
        color='blue',
        weight=2,
        opacity=0.9
    )

    # Add the polyline to the map
    polyline_obj.add_to(m)

# Save the map as an HTML file
map_file = os.path.join('images', 'ukraine-route-map.html')
m.save(map_file)
print(f"Map saved as {map_file}")

# Display the map
m

In [ ]:
import os
import pandas as pd
import numpy as np
import requests
import folium
import polyline
from sklearn.neighbors import KDTree
from scipy.spatial import distance_matrix
from scipy.optimize import minimize
from scipy.interpolate import splprep, splev

# Set the file name of the locations.csv file
locations_csv = 'ukraine-locations.csv'

# Set the path to the directory where route.geojson is stored
locations_csv_path = '.'

# Check if the CSV file exists
locations_csv_file = os.path.join(locations_csv_path, locations_csv)
if not os.path.exists(locations_csv_file):
    print(f"Error: {locations_csv} file not found.")
    exit(1)

# Read the locations from CSV
df = pd.read_csv(locations_csv_file)

# Create a map centered around the first location
start_lat = df.loc[0, 'latitude']
start_lon = df.loc[0, 'longitude']
m = folium.Map(location=[start_lat, start_lon], zoom_start=6)

# Create an empty list to store route instructions
route_instructions = []

# Create an empty list to store coordinates for clustering
all_coordinates = []

# Iterate through each pair of locations in the DataFrame
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        # Extract location details
        name1 = df.loc[i, 'name']
        lat1 = df.loc[i, 'latitude']
        lon1 = df.loc[i, 'longitude']
        name2 = df.loc[j, 'name']
        lat2 = df.loc[j, 'latitude']
        lon2 = df.loc[j, 'longitude']

        # Prepare API request URL
        osrm_endpoint = 'http://localhost:5000/route/v1/driving'
        request_url = f"{osrm_endpoint}/{lon1},{lat1};{lon2},{lat2}?steps=true&geometries=polyline"

        try:
            # Send request to OSRM
            response = requests.get(request_url)
            response.raise_for_status()

            # Extract route data from the response
            route_data = response.json()

            # Check if response contains a route
            if 'routes' in route_data and len(route_data['routes']) > 0:
                # Extract route steps
                steps = route_data['routes'][0]['legs'][0]['steps']
                distance_sum = 0
                for step in steps:
                    # Extract step instructions
                    instruction = step['name']

                    # Extract step distance and add to the distance sum
                    distance = step['distance']
                    distance_sum += distance

                # Append step information to the route_instructions list
                route_instructions.append((instruction, distance_sum))

                # Extract route geometry
                geometry = route_data['routes'][0]['geometry']

                # Decode polyline string to get coordinates
                coordinates = polyline.decode(geometry)

                # Add the coordinates to the list for clustering
                all_coordinates.extend(coordinates)

                # Add markers for the start and end locations
                folium.CircleMarker(
                    location=[lat1, lon1],
                    radius=6,
                    color='red',
                    fill=True,
                    fill_color='blue',
                    popup=name1
                ).add_to(m)

                folium.CircleMarker(
                    location=[lat2, lon2],
                    radius=6,
                    color='red',
                    fill=True,
                    fill_color='blue',
                    popup=name2
                ).add_to(m)

        except requests.exceptions.RequestException as e:
            print(f"Error occurred during OSRM API request: {e}")
            continue

# Perform KD-Tree clustering for coordinates
kdtree = KDTree(all_coordinates)
eps = 0.01  # maximum distance between two points to be considered in the same cluster
clusters = kdtree.query_radius(all_coordinates, eps)

# Convert all_coordinates to a NumPy array
all_coordinates = np.array(all_coordinates)

# Create a list to store the final cluster coordinates
final_cluster_coords = []

# Iterate over the clusters and generate polylines for each cluster
for cluster in clusters:
    # Check the number of points in the cluster, and if it's less than 8 (for cubic B-spline), skip interpolation
    if len(cluster) < 8:
        # Print a message or perform other actions if needed
        print("Skipping interpolation - Not enough points in the cluster.")
        continue

    # Concatenate the arrays in the cluster list to get the coordinates of the current cluster
    cluster_coords = all_coordinates[cluster]

    # Calculate the distance matrix for the cluster coordinates
    distance_matrix_cluster = distance_matrix(cluster_coords, cluster_coords)

    # Define the objective function for TSP
    def tsp_objective_function(order):
        # Cast float values to integers for indexing
        order = order.astype(int)
        return sum([distance_matrix_cluster[order[i]][order[i + 1]] for i in range(len(order) - 1)])

    # Define the constraint function for TSP
    def tsp_constraint_function(order):
        # Cast float values to integers for indexing
        order = order.astype(int)
        return sum(order) - len(order)

    # Define the initial guess for the TSP
    initial_guess = list(range(len(cluster_coords)))

    # Define the bounds for the TSP
    bounds = [(0, len(cluster_coords) - 1)] * len(cluster_coords)

    # Define the constraint for the TSP
    constraint = {'type': 'eq', 'fun': tsp_constraint_function}

    # Solve the TSP using the minimize function
    result = minimize(tsp_objective_function, initial_guess, method='SLSQP', bounds=bounds, constraints=constraint)

    # Get the optimal order of locations from the result and convert to integers
    optimal_order = result.x.astype(int)

    # Reorder the cluster coordinates based on the optimal order
    reordered_coords = [cluster_coords[i] for i in optimal_order]

    # Convert reordered_coords to a numpy array for spline interpolation
    route_coords_np = np.array(reordered_coords)
    
    try:
        # Perform spline interpolation to generate additional points
        tck, _ = splprep(route_coords_np.T, s=0.0, per=False)

        # Define the number of additional points you want to generate
        num_additional_points = 100

        # Generate additional points using splev
        u_new = np.linspace(0, 1, num_additional_points)
        new_points = splev(u_new, tck)

        # Append the new_points to the cluster_coords
        cluster_coords = np.vstack((cluster_coords, np.array(new_points).T))
        
    except ValueError:
        # If interpolation fails, print a message and use a fallback method
        print("Spline interpolation failed. Using fallback method.")
        # Fallback: Connect points directly with straight lines
        cluster_coords = np.array(cluster_coords)

    # Add the extended cluster coordinates to the list
    final_cluster_coords.extend(cluster_coords)

# Create a list of polyline coordinates
polyline_coords = [(coord[0], coord[1]) for coord in final_cluster_coords]

# Create a Folium PolyLine object for the entire route
polyline_obj = folium.PolyLine(
    locations=polyline_coords,
    color='blue',
    weight=2,
    opacity=0.9
)

# Add the polyline to the map
polyline_obj.add_to(m)

# Save the map as an HTML file
map_file = os.path.join('images', 'ukraine-route-map.html')
m.save(map_file)
print(f"Map saved as {map_file}")

# Display the map
m

In [ ]:
import os
import pandas as pd
import numpy as np
import requests
import folium
import polyline
from sklearn.cluster import DBSCAN
from scipy.spatial import distance_matrix
from scipy.optimize import minimize

# Set the file name of the locations.csv file
locations_csv = 'ukraine-locations.csv'

# Set the path to the directory where route.geojson is stored
locations_csv_path = '.'

# Check if the CSV file exists
locations_csv_file = os.path.join(locations_csv_path, locations_csv)
if not os.path.exists(locations_csv_file):
    print(f"Error: {locations_csv} file not found.")
    exit(1)

# Read the locations from CSV
df = pd.read_csv(locations_csv_file)

# Create a map centered around the first location
start_lat = df.loc[0, 'latitude']
start_lon = df.loc[0, 'longitude']
m = folium.Map(location=[start_lat, start_lon], zoom_start=6)

# Create an empty list to store route instructions
route_instructions = []

# Create an empty list to store coordinates for clustering
all_coordinates = []

# Iterate through each pair of locations in the DataFrame
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        # Extract location details
        name1 = df.loc[i, 'name']
        lat1 = df.loc[i, 'latitude']
        lon1 = df.loc[i, 'longitude']
        name2 = df.loc[j, 'name']
        lat2 = df.loc[j, 'latitude']
        lon2 = df.loc[j, 'longitude']

        # Prepare API request URL
        osrm_endpoint = 'http://localhost:5000/route/v1/driving'
        request_url = f"{osrm_endpoint}/{lon1},{lat1};{lon2},{lat2}?steps=true&geometries=polyline"

        try:
            # Send request to OSRM
            response = requests.get(request_url)
            response.raise_for_status()

            # Extract route data from the response
            route_data = response.json()

            # Check if response contains a route
            if 'routes' in route_data and len(route_data['routes']) > 0:
                # Extract route steps
                steps = route_data['routes'][0]['legs'][0]['steps']
                distance_sum = 0
                for step in steps:
                    # Extract step instructions
                    instruction = step['name']

                    # Extract step distance and add to the distance sum
                    distance = step['distance']
                    distance_sum += distance

                # Append step information to the route_instructions list
                route_instructions.append((instruction, distance_sum))

                # Extract route geometry
                geometry = route_data['routes'][0]['geometry']

                # Decode polyline string to get coordinates
                coordinates = polyline.decode(geometry)

                # Add the coordinates to the list for clustering
                all_coordinates.extend(coordinates)

                # Add markers for the start and end locations
                folium.CircleMarker(
                    location=[lat1, lon1],
                    radius=6,
                    color='red',
                    fill=True,
                    fill_color='blue',
                    popup=name1
                ).add_to(m)

                folium.CircleMarker(
                    location=[lat2, lon2],
                    radius=6,
                    color='red',
                    fill=True,
                    fill_color='blue',
                    popup=name2
                ).add_to(m)

        except requests.exceptions.RequestException as e:
            print(f"Error occurred during OSRM API request: {e}")
            continue

# Perform DBSCAN (Density-Based Spatial Clustering of Applications with Noise) for coordinates
eps = 0.001  # distance threshold for clustering
min_samples = 2  # minimum number of points in a cluster
dbscan = DBSCAN(eps=eps, min_samples=min_samples)
cluster_labels = dbscan.fit_predict(all_coordinates)

# Create a dictionary to store polylines for each cluster
cluster_polylines = {}

# Create a list to store the final cluster coordinates
final_cluster_coords = []

# Iterate over the clusters and generate polylines for each cluster
for label in set(cluster_labels):
    if label == -1:
        # Skip noise points (not assigned to any cluster)
        continue

    # Extract coordinates for the current cluster
    cluster_coords = [coord for i, coord in enumerate(all_coordinates) if cluster_labels[i] == label]

    # Append the cluster_coords to the final_cluster_coords
    final_cluster_coords.append(cluster_coords)

    # Create a list of polyline coordinates
    polyline_coords = [(coord[0], coord[1]) for coord in cluster_coords]
    
    polyline_coords = set(polyline_coords)
    
    if len(polyline_coords) < 3:
        continue
    
    # Create a Folium PolyLine object for the current cluster
    polyline_obj = folium.PolyLine(
        locations=polyline_coords,
        color='blue',
        weight=2,
        opacity=0.9
    )

    # Add the polyline to the map
    polyline_obj.add_to(m)

# Save the map as an HTML file
map_file = os.path.join('images', 'ukraine-route-map.html')
m.save(map_file)
print(f"Map saved as {map_file}")

# Display the map
m

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def euclidean_distance(a, b):
    return np.linalg.norm(np.array(a) - np.array(b))

def nearest_neighbor(locations):
    num_locations = len(locations)
    unvisited = set(range(num_locations))
    route = []
    
    # Choose a random starting location
    current_location = np.random.choice(num_locations)
    route.append(current_location)
    unvisited.remove(current_location)
    
    while unvisited:
        nearest_dist = float('inf')
        nearest_location = None
        
        for loc in unvisited:
            dist = euclidean_distance(locations[current_location], locations[loc])
            if dist < nearest_dist:
                nearest_dist = dist
                nearest_location = loc
        
        current_location = nearest_location
        route.append(current_location)
        unvisited.remove(current_location)
    
    return route

def visualize_route(locations, route):
    x = [loc[0] for loc in locations]
    y = [loc[1] for loc in locations]

    plt.figure(figsize=(8, 6))
    plt.scatter(x, y, c='blue', s=100, zorder=2)
    for i in range(len(route) - 1):
        start = route[i]
        end = route[i + 1]
        plt.plot([x[start], x[end]], [y[start], y[end]], 'r-', zorder=1)
    plt.plot([x[route[-1]], x[route[0]]], [y[route[-1]], y[route[0]]], 'r-', zorder=1)
    plt.xlabel('X-coordinate')
    plt.ylabel('Y-coordinate')
    plt.title('Nearest Neighbor Algorithm - Shortest Route')
    plt.grid(True)
    plt.show()

if __name__ == "__main__":
    # Replace these coordinates with your actual locations A, B, C, D, etc.
    locations = [(0, 0), (1, 2), (4, 3), (2, 5)]
    route = nearest_neighbor(locations)
    print("Optimal route:", route)
    visualize_route(locations, route)


In [ ]:
import os
import csv
import pandas as pd
import requests
import folium
import polyline
import numpy as np

region = "ukraine"

# Set the file name of the locations.csv file
locations_csv = f'{region}-locations.csv'

# Set the path to the directory where route.geojson is stored
locations_csv_path = '.'

# Check if the CSV file exists
locations_csv_file = os.path.join(locations_csv_path, locations_csv)
if not os.path.exists(locations_csv_file):
    print(f"Error: {locations_csv} file not found.")
    exit(1)

# Read the locations from CSV
df = pd.read_csv(locations_csv_file)

# Create a new DataFrame with only the required columns
locations = df[['name', 'latitude', 'longitude']]

def euclidean_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

def nearest_neighbor_route(locations):
    # Create a list to keep track of visited locations
    visited = []
    unvisited = list(range(len(locations)))  # List of indices of unvisited locations

    # Start from the first location (index 0)
    current_location = 0
    route = [current_location]

    # Loop through all the locations
    while len(unvisited) > 1:
        nearest_dist = float('inf')
        nearest_location = None

        for loc in unvisited:
            if loc != current_location:  # Skip the current location itself
                dist = euclidean_distance(locations.iloc[current_location][['latitude', 'longitude']],
                                          locations.iloc[loc][['latitude', 'longitude']])
                if dist < nearest_dist:
                    nearest_dist = dist
                    nearest_location = loc

        # Mark the nearest location as visited
        visited.append(current_location)

        # Move to the nearest location and remove it from the unvisited list
        current_location = nearest_location
        unvisited.remove(nearest_location)

        # Add the nearest location to the route
        route.append(current_location)

    # Add the first location (index 0) at the end to complete the route
    route.append(0)

    return route


# Reset the index of the DataFrame
df = df.reset_index(drop=True)

# Get the optimal route using the nearest neighbor algorithm
route = nearest_neighbor_route(df)

# Create a map centered around the first location
start_lat = df.loc[route[0], 'latitude']
start_lon = df.loc[route[0], 'longitude']
m = folium.Map(location=[start_lat, start_lon], zoom_start=6)

# Create an empty list to store route instructions
route_instructions = []

# Create an empy list to store all routes
routes = []

# Iterate through the optimal route
for i in range(len(route) - 1):
    # Extract location details
    name1 = df.loc[route[i], 'name']
    lat1 = df.loc[route[i], 'latitude']
    lon1 = df.loc[route[i], 'longitude']
    name2 = df.loc[route[i + 1], 'name']
    lat2 = df.loc[route[i + 1], 'latitude']
    lon2 = df.loc[route[i + 1], 'longitude']

    # Prepare API request URL
    osrm_endpoint = 'http://localhost:5000/route/v1/driving'
    request_url = f"{osrm_endpoint}/{lon1},{lat1};{lon2},{lat2}?steps=true&geometries=polyline"

    try:
        # Send request to OSRM
        response = requests.get(request_url)
        response.raise_for_status()

        # Extract route data from the response
        route_data = response.json()

        # Check if response contains a route
        if 'routes' in route_data and len(route_data['routes']) > 0:
            # Extract route steps
            steps = route_data['routes'][0]['legs'][0]['steps']
            distance_sum = 0
            for step in steps:
                # Extract step instructions
                instruction = step['name']

                # Extract step distance and add to the distance sum
                distance = step['distance']
                distance_sum += distance

                # Append step information to the route_instructions list
                route_instructions.append((instruction, distance_sum))

            # Extract route geometry
            geometry = route_data['routes'][0]['geometry']

            # Decode polyline string to get coordinates
            coordinates = polyline.decode(geometry)
            
            routes.append(coordinates)

            # Create a Folium PolyLine object
            polyline_obj = folium.PolyLine(
                locations=coordinates,
                color='blue',
                weight=2,
                opacity=0.9
            )

            # Add the PolyLine to the map
            polyline_obj.add_to(m)

    except requests.exceptions.RequestException as e:
        print(f"Error occurred during OSRM API request: {e}")
        continue

    # Add markers for the start and end locations
    folium.CircleMarker(
        location=[lat1, lon1],
        radius=6,
        color='red',
        fill=True,
        fill_color='blue',
        popup=name1
    ).add_to(m)

    folium.CircleMarker(
        location=[lat2, lon2],
        radius=6,
        color='red',
        fill=True,
        fill_color='blue',
        popup=name2
    ).add_to(m)


# CSV file path
csv_file = f"{region}-route-coords.csv"

# Write the nested list to the CSV file
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["latitude", "longitude"])  # Write the header
    for route in routes:
        writer.writerows(route)  # Write each sublist of coordinates to the CSV file

# Save the map as an HTML file
map_file = os.path.join('images', f'{region}-route-map.html')
m.save(map_file)
print(f"Map saved as {map_file}")

# Display the map
m


In [ ]:
import os
import pandas as pd
import requests

region = 'ukraine'

# Set the file name of the locations.csv file
locations_csv = f'{region}-locations.csv'

# Set the path to the directory where route.geojson is stored
locations_csv_path = '.'

# Check if the CSV file exists
if not os.path.exists(os.path.join(locations_csv_path, locations_csv)):
    print(f"Error: {locations_csv} file not found.")
    exit(1)

# Read the locations from CSV
df = pd.read_csv(os.path.join(locations_csv_path, locations_csv))

# Create an empty DataFrame to store the distances
num_locations = len(df)
distance_matrix = pd.DataFrame(index=range(num_locations), columns=df['name'])
distance_matrix = distance_matrix.fillna(0)

# Iterate through each pair of locations in the DataFrame
for i in range(num_locations):
    for j in range(i + 1, num_locations):
        # Extract location details
        lat1 = df.loc[i, 'latitude']
        lon1 = df.loc[i, 'longitude']
        lat2 = df.loc[j, 'latitude']
        lon2 = df.loc[j, 'longitude']

        # Prepare API request URL
        osrm_endpoint = 'http://localhost:5000/route/v1/driving'
        request_url = f"{osrm_endpoint}/{lon1},{lat1};{lon2},{lat2}?steps=true&geometries=polyline"

        # Send request to OSRM
        response = requests.get(request_url)
        route_data = response.json()

        # Check if response contains a route
        if 'routes' in route_data and len(route_data['routes']) > 0:
            # Extract distance
            distance = route_data['routes'][0]['distance']

            # Update the distance matrix
            distance_matrix.at[i, df.loc[j, 'name']] = distance
            distance_matrix.at[j, df.loc[i, 'name']] = distance

# Save the distance matrix to a CSV file
distance_matrix.to_csv(f'{region}-distances.csv', index_label='name')

print(f"Distances saved as {region}-distances.csv")


In [ ]:
import numpy as np
import pandas as pd
from itertools import combinations
from geopy.distance import geodesic


def geo_pruning(x, df, k, b):
    name = x.name
    distances = []
    x1 = df.loc[df['name'] == name, 'latitude'].values[0]
    y1 = df.loc[df['name'] == name, 'longitude'].values[0]
    for xx in df['name']:
        x2 = df.loc[df['name'] == xx, 'latitude'].values[0]
        y2 = df.loc[df['name'] == xx, 'longitude'].values[0]
        distance = geodesic((x1, y1), (x2, y2)).meters
        distances.append(distance)
    knn = sorted(distances)[k]
    distances = [d if d <= knn * b else 0 for d in distances]
    return distances


def knnroute(x, k):
    knn = x.nsmallest(k).iloc[-1]
    return x.where(x <= knn)


def rSubset(arr, r):
    return list(combinations(arr, r))


def trianglepruning(x, triangle_factor):
    x2 = pd.DataFrame(columns=x.columns, index=x.index)
    for num, column in enumerate(list(x.columns)[2:]):
        knns = list(x[column][x[column].notna()].index)
        knn_distances = list(x[column][x[column].notna()])
        dictionary = dict(zip(knns, knn_distances))
        knn_combinations = rSubset(knns, 2)
        for comb in knn_combinations:
            c = x.loc[comb[0], comb[1]]
            a = dictionary[comb[0]]
            b = dictionary[comb[1]]
            triangle = [a, b, c]
            longest = triangle.pop(triangle.index(max(triangle)))
            if longest > triangle_factor * (sum(triangle)):
                if longest == a:
                    x2.loc[comb[0], column] = 1
                    # print(str(comb[0]) + ' - ' + column + ' via ' + str(comb[1]))
                elif longest == b:
                    x2.loc[comb[1], column] = 1
                    # print(str(comb[1]) + ' - ' + column + ' via ' + str(comb[0]))
                elif longest == c:
                    x2.loc[comb[0], comb[1]] = 1
                    # print(str(comb[0]) + ' - ' + str(comb[1]) + ' via ' + column)
    return x.where(x2.isna().astype(bool))


def save_to_csv(x, filename):
    res = x[x != 0].stack().reset_index()
    res = res.rename({'Index': 'name1', 'name': 'name2', 0: 'distance'}, axis=1)
    res.distance = res.distance / 1000
    res.to_csv(filename, index=False)

region = 'ukraine'

Locations = pd.read_csv(f'{region}-locations.csv')

Distances = pd.read_csv(f'{region}-distances.csv', index_col='name')

Distances_test = Distances
Distances_test['Index'] = list(Distances.columns)
Distances_test.set_index('Index', inplace=True)

Distances.to_csv(f"{region}-myfile.csv")

triangle_factors = np.arange(0.8, 1., 0.01)
for triangle_factor in triangle_factors:
    Distances_Triangle = Distances
    Distances_Triangle = Distances_Triangle.replace(0, np.NaN)
    Distances_Triangle['Index'] = list(Distances.columns)
    Distances_Triangle.set_index('Index', inplace=True)
    Distances_Triangle = Distances_Triangle.where(np.triu(np.ones(Distances_Triangle.shape)).astype(bool))
    Distances_Triangle = trianglepruning(Distances_Triangle, triangle_factor);
    save_to_csv(Distances_Triangle, 'Distances_Triangle_%.3f.csv' % triangle_factor)

# print(triangle_factors)

Distances_knn = Distances.replace(0, np.NaN)
Distances_knn['Index'] = list(Distances.columns)
Distances_knn.set_index('Index', inplace=True)
Distances_knn = Distances_knn.apply(knnroute, args=(9,))
save_to_csv(Distances_knn, 'Distances_knn.csv')
# Distances_knn.head()

Distances_geo = Distances.apply(geo_pruning, args=(Locations, 6, 1.1,))
Distances_geo = Distances.where(Distances_geo > 0)
Distances_geo = Distances_geo.replace(to_replace=40000000, value=np.nan)
Distances_geo['Index'] = list(Distances.columns)
Distances_geo.set_index('Index', inplace=True)
save_to_csv(Distances_geo, 'Distances_geo.csv')
# Distances_geo.head()

Distances_geo_triangle = pd.DataFrame(np.maximum(
    Distances_geo.values, 
    Distances_geo.values.T), 
    index=Distances_geo.index, 
    columns=Distances_geo.columns)

Distances_geo_triangle = Distances_geo_triangle.replace(0, np.NaN)
Distances_geo_triangle['Index'] = list(Distances_geo.columns)
Distances_geo_triangle.set_index('Index', inplace=True)
Distances_geo_triangle = Distances_geo_triangle.where(np.triu(np.ones(Distances_geo.shape)).astype(np.bool_))
Distances_geo_triangle = trianglepruning(Distances_geo_triangle, 0.8)
save_to_csv(Distances_geo_triangle, 'Distances_geo_triangle.csv')
# Distances_geo_triangle.head()

In [ ]:
import requests
import pandas as pd

# Function to read the pruned distance matrix from CSV and create a list of dictionaries
def read_distance_matrix(filename):
    distances = pd.read_csv(filename)
    distance_list = []
    for _, row in distances.iterrows():
        distance_dict = {
            "name1": row["name1"],
            "name2": row["level_1"],
            "distance": row["distance"]
        }
        distance_list.append(distance_dict)
    return distance_list

# Replace 'your_osrm_endpoint' with the URL of your custom OSRM engine
osrm_endpoint = 'http://localhost:5000/route/v1/driving'

# Load pruned distance matrices for different pruning techniques
distances_triangle = read_distance_matrix('Distances_Triangle_0.800.csv')
distances_knn = read_distance_matrix('Distances_knn.csv')
distances_geo = read_distance_matrix('Distances_geo.csv')
distances_geo_triangle = read_distance_matrix('Distances_geo_triangle.csv')

# Function to perform routing analysis with the custom OSRM endpoint using a given distance matrix
def perform_routing_analysis(distance_matrix):
    routes = []
    for entry in distance_matrix:
        name1 = entry["name1"]
        name2 = entry["name2"]
        distance = entry["distance"]

        # Query the custom OSRM endpoint for the route between name1 and name2
        request_url = f"{osrm_endpoint}/{Locations[Locations['name'] == name1]['longitude'].iloc[0]},{Locations[Locations['name'] == name1]['latitude'].iloc[0]};{Locations[Locations['name'] == name2]['longitude'].iloc[0]},{Locations[Locations['name'] == name2]['latitude'].iloc[0]}?steps=true&geometries=polyline"
        response = requests.get(request_url)

        # Check if the request was successful
        if response.status_code == 200:
            route_info = response.json().get('routes', [])
            if route_info:
                route = {
                    "name1": name1,
                    "name2": name2,
                    "distance": distance,
                    "duration": route_info[0]["duration"],
                    "distance_osrm": route_info[0]["distance"],
                    "geometry": route_info[0]["geometry"]
                }
                routes.append(route)
        else:
            print(f"Failed to fetch route for {name1} and {name2}")

    return routes

# Perform routing analysis with different distance matrices
routes_triangle = perform_routing_analysis(distances_triangle)
routes_knn = perform_routing_analysis(distances_knn)
routes_geo = perform_routing_analysis(distances_geo)
routes_geo_triangle = perform_routing_analysis(distances_geo_triangle)

# Now you have the routes and their corresponding distance matrices from the custom OSRM endpoint.
# You can use this information to analyze the differences between the original distances and the pruned distances,
# as well as to optimize the routing based on the pruned data.

# Note: Make sure to replace 'your_osrm_endpoint' with the actual URL of your custom OSRM endpoint.


In [ ]:
import os
import pandas as pd
import numpy as np
import requests
import folium
import polyline
from itertools import combinations
from geopy.distance import geodesic


def geo_pruning(x, df, k, b):
    name = x.name
    distances = []
    x1 = df.loc[df['name'] == name, 'latitude'].values[0]
    y1 = df.loc[df['name'] == name, 'longitude'].values[0]
    for xx in df['name']:
        x2 = df.loc[df['name'] == xx, 'latitude'].values[0]
        y2 = df.loc[df['name'] == xx, 'longitude'].values[0]
        distance = geodesic((x1, y1), (x2, y2)).meters
        distances.append(distance)
    knn = sorted(distances)[k]
    distances = [d if d <= knn * b else 0 for d in distances]
    return distances


def knnroute(x, k):
    knn = x.nsmallest(k).iloc[-1]
    return x.where(x <= knn)


def rSubset(arr, r):
    return list(combinations(arr, r))


def trianglepruning(x, triangle_factor):
    x2 = pd.DataFrame(columns=x.columns, index=x.index)
    for num, column in enumerate(list(x.columns)[2:]):
        knns = list(x[column][x[column].notna()].index)
        knn_distances = list(x[column][x[column].notna()])
        dictionary = dict(zip(knns, knn_distances))
        knn_combinations = rSubset(knns, 2)
        for comb in knn_combinations:
            c = x.loc[comb[0], comb[1]]
            a = dictionary[comb[0]]
            b = dictionary[comb[1]]
            triangle = [a, b, c]
            longest = triangle.pop(triangle.index(max(triangle)))
            if longest > triangle_factor * (sum(triangle)):
                if longest == a:
                    x2.loc[comb[0], column] = 1
                    # print(str(comb[0]) + ' - ' + column + ' via ' + str(comb[1]))
                elif longest == b:
                    x2.loc[comb[1], column] = 1
                    # print(str(comb[1]) + ' - ' + column + ' via ' + str(comb[0]))
                elif longest == c:
                    x2.loc[comb[0], comb[1]] = 1
                    # print(str(comb[0]) + ' - ' + str(comb[1]) + ' via ' + column)
    return x.where(x2.isna().astype(bool))


def save_to_csv(x, filename):
    res = x[x != 0].stack().reset_index()
    res = res.rename({'Index': 'name1', 'name': 'name2', 0: 'distance'}, axis=1)
    res.distance = res.distance / 1000
    res.to_csv(filename, index=False)
    
def get_polyline_string(origin, destination):
    url = f"http://router.project-osrm.org/route/v1/driving/{origin[1]},{origin[0]};{destination[1]},{destination[0]}"
    response = requests.get(url)
    data = response.json()
    if response.status_code == 200 and data.get("code") == "Ok":
        route = data.get("routes")[0]
        polyline_string = route.get("geometry")
        return polyline_string
    else:
        return None

def visualize_routes_on_map(locations_df, routes_df, region):
    # Create a map centered around the first location
    start_lat = locations_df.loc[0, 'latitude']
    start_lon = locations_df.loc[0, 'longitude']
    m = folium.Map(location=[start_lat, start_lon], zoom_start=6)

    # Add markers for each location
    for i, row in locations_df.iterrows():
        name = row['name']
        lat = row['latitude']
        lon = row['longitude']
        folium.CircleMarker(
            location=[lat, lon],
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name
        ).add_to(m)

    # Iterate through the routes_df DataFrame and update the geometry column
    for i, row in routes_df.iterrows():
        name1 = row['name1']
        name2 = row['name2']
        origin = (locations_df.loc[locations_df['name'] == name1, 'latitude'].values[0],
                  locations_df.loc[locations_df['name'] == name1, 'longitude'].values[0])
        destination = (locations_df.loc[locations_df['name'] == name2, 'latitude'].values[0],
                       locations_df.loc[locations_df['name'] == name2, 'longitude'].values[0])
        geometry_str = get_polyline_string(origin, destination)
        routes_df.at[i, 'geometry'] = geometry_str

        # Skip rows with NaN 'geometry' values
        if pd.isna(geometry_str):
            print(f"Warning: No route between {name1} and {name2}, skipping.")
            continue

        # Attempt to decode the polyline
        try:
            coordinates = polyline.decode(geometry_str)
        except ValueError:
            print(f"Warning: Invalid 'geometry' value for row {i}, skipping.")
            continue

        # Continue with existing code
        polyline_obj = folium.PolyLine(
            locations=coordinates,
            color='blue',
            weight=2,
            opacity=0.9,
            popup=f"{name1} to {name2}"
        )
        polyline_obj.add_to(m)

        
    # Save the map as an HTML file
    map_file = f'{region}-route-map.html'  # Customize the file name as needed
    m.save(map_file)
    print(f"Map saved as {map_file}")

    # Display the map
    return m


region = 'ukraine'

Locations = pd.read_csv(f'{region}-locations.csv')
Distances = pd.read_csv(f'{region}-distances.csv', index_col='name')

Distances_test = Distances
Distances_test['Index'] = list(Distances.columns)
Distances_test.set_index('Index', inplace=True)

Distances.to_csv("myfile.csv")

triangle_factors = np.arange(0.8, 1., 0.01)
for triangle_factor in triangle_factors:
    Distances_Triangle = Distances
    Distances_Triangle = Distances_Triangle.replace(0, np.NaN)
    Distances_Triangle['Index'] = list(Distances.columns)
    Distances_Triangle.set_index('Index', inplace=True)
    Distances_Triangle = Distances_Triangle.where(np.triu(np.ones(Distances_Triangle.shape)).astype(bool))
    Distances_Triangle = trianglepruning(Distances_Triangle, triangle_factor);
    save_to_csv(Distances_Triangle, 'Distances_Triangle_%.3f.csv' % triangle_factor)

Distances_knn = Distances.replace(0, np.NaN)
Distances_knn['Index'] = list(Distances.columns)
Distances_knn.set_index('Index', inplace=True)
Distances_knn = Distances_knn.apply(knnroute, args=(9,))
save_to_csv(Distances_knn, 'Distances_knn.csv')
Distances_knn.head()

# Apply geo_pruning function
Distances_geo = Distances.apply(geo_pruning, args=(Locations, 6, 1.1,))
Distances_geo = Distances.where(Distances_geo > 0)
Distances_geo = Distances_geo.replace(to_replace=40000000, value=np.nan)
Distances_geo['Index'] = list(Distances.columns)
Distances_geo.set_index('Index', inplace=True)
save_to_csv(Distances_geo, 'Distances_geo.csv')
Distances_geo.head()

Distances_geo_triangle = pd.DataFrame(np.maximum(Distances_geo.values, Distances_geo.values.T),
                                      index=Distances_geo.index, columns=Distances_geo.columns)
Distances_geo_triangle = Distances_geo_triangle.replace(0, np.NaN)
Distances_geo_triangle['Index'] = list(Distances_geo.columns)
Distances_geo_triangle.set_index('Index', inplace=True)
Distances_geo_triangle = Distances_geo_triangle.where(np.triu(np.ones(Distances_geo.shape)).astype(np.bool_))
Distances_geo_triangle = trianglepruning(Distances_geo_triangle, 0.8)
save_to_csv(Distances_geo_triangle, 'Distances_geo_triangle.csv')
Distances_geo_triangle.head()

# Save the pruned distances to CSV files
save_to_csv(Distances_Triangle, 'Distances_Triangle_0.800.csv')
save_to_csv(Distances_knn, 'Distances_knn.csv')
save_to_csv(Distances_geo, 'Distances_geo.csv')
save_to_csv(Distances_geo_triangle, 'Distances_geo_triangle.csv')

# Create the routes DataFrame from the pruned distances
routes_list = []
for i, row in Distances_Triangle.iterrows():
    for col in Distances_Triangle.columns:
        distance = row[col]
        if not pd.isna(distance):
            # Ensure that 'geometry' is a valid number (not NaN) before appending to routes_list
            geometry = Distances_geo.loc[i, col]
            if not pd.isna(geometry):
                origin = (Locations.loc[Locations['name'] == i, 'latitude'].values[0],
                          Locations.loc[Locations['name'] == i, 'longitude'].values[0])
                destination = (Locations.loc[Locations['name'] == col, 'latitude'].values[0],
                               Locations.loc[Locations['name'] == col, 'longitude'].values[0])
                geometry_str = get_polyline_string(origin, destination)
                if geometry_str is not None:
                    routes_list.append({
                        'name1': i,
                        'name2': col,
                        'geometry': geometry_str
                    })
                else:
                    print(f"Warning: No route between {i} and {col}, skipping.")

# Concatenate the list of dictionaries into a DataFrame
routes_df = pd.DataFrame(routes_list)

# Visualize the routes on the map
m = visualize_routes_on_map(Locations, routes_df, region)

m

In [ ]:
import os
import pandas as pd
import numpy as np
import requests
import folium
import polyline
from itertools import combinations
from geopy.distance import geodesic


def geo_pruning(x, df, k, b):
    name = x.name
    distances = []
    x1 = df.loc[df['name'] == name, 'latitude'].values[0]
    y1 = df.loc[df['name'] == name, 'longitude'].values[0]
    for xx in df['name']:
        x2 = df.loc[df['name'] == xx, 'latitude'].values[0]
        y2 = df.loc[df['name'] == xx, 'longitude'].values[0]
        distance = geodesic((x1, y1), (x2, y2)).meters
        distances.append(distance)
    knn = sorted(distances)[k]
    distances = [d if d <= knn * b else 0 for d in distances]
    return distances


def knnroute(x, k):
    knn = x.nsmallest(k).iloc[-1]
    return x.where(x <= knn)


def rSubset(arr, r):
    return list(combinations(arr, r))


def trianglepruning(x, triangle_factor):
    x2 = pd.DataFrame(columns=x.columns, index=x.index)
    for num, column in enumerate(list(x.columns)[2:]):
        knns = list(x[column][x[column].notna()].index)
        knn_distances = list(x[column][x[column].notna()])
        dictionary = dict(zip(knns, knn_distances))
        knn_combinations = rSubset(knns, 2)
        for comb in knn_combinations:
            c = x.loc[comb[0], comb[1]]
            a = dictionary[comb[0]]
            b = dictionary[comb[1]]
            triangle = [a, b, c]
            longest = triangle.pop(triangle.index(max(triangle)))
            if longest > triangle_factor * (sum(triangle)):
                if longest == a:
                    x2.loc[comb[0], column] = 1
                    # print(str(comb[0]) + ' - ' + column + ' via ' + str(comb[1]))
                elif longest == b:
                    x2.loc[comb[1], column] = 1
                    # print(str(comb[1]) + ' - ' + column + ' via ' + str(comb[0]))
                elif longest == c:
                    x2.loc[comb[0], comb[1]] = 1
                    # print(str(comb[0]) + ' - ' + str(comb[1]) + ' via ' + column)
    return x.where(x2.isna().astype(bool))


def save_to_csv(x, filename):
    res = x[x != 0].stack().reset_index()
    res = res.rename({'Index': 'name1', 'name': 'name2', 0: 'distance'}, axis=1)
    res.distance = res.distance / 1000
    res.to_csv(filename, index=False)
    
def get_polyline_string(origin, destination):
    url = f"http://router.project-osrm.org/route/v1/driving/{origin[1]},{origin[0]};{destination[1]},{destination[0]}"
    response = requests.get(url)
    data = response.json()
    if response.status_code == 200 and data.get("code") == "Ok":
        route = data.get("routes")[0]
        polyline_string = route.get("geometry")
        return polyline_string
    else:
        return None

def visualize_routes_on_map(locations_df, routes_df, region, triangle_factor):
    # Create a map centered around the first location
    start_lat = locations_df.loc[0, 'latitude']
    start_lon = locations_df.loc[0, 'longitude']
    m = folium.Map(location=[start_lat, start_lon], zoom_start=6)

    # Add markers for each location
    for i, row in locations_df.iterrows():
        name = row['name']
        lat = row['latitude']
        lon = row['longitude']
        folium.CircleMarker(
            location=[lat, lon],
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name
        ).add_to(m)

    # Iterate through the routes_df DataFrame and update the geometry column
    for i, row in routes_df.iterrows():
        name1 = row['name1']
        name2 = row['name2']
        origin = (locations_df.loc[locations_df['name'] == name1, 'latitude'].values[0],
                  locations_df.loc[locations_df['name'] == name1, 'longitude'].values[0])
        destination = (locations_df.loc[locations_df['name'] == name2, 'latitude'].values[0],
                       locations_df.loc[locations_df['name'] == name2, 'longitude'].values[0])
        geometry_str = get_polyline_string(origin, destination)
        routes_df.at[i, 'geometry'] = geometry_str

        # Skip rows with NaN 'geometry' values
        if pd.isna(geometry_str):
            print(f"Warning: No route between {name1} and {name2}, skipping.")
            continue

        # Attempt to decode the polyline
        try:
            coordinates = polyline.decode(geometry_str)
        except ValueError:
            print(f"Warning: Invalid 'geometry' value for row {i}, skipping.")
            continue

        # Continue with existing code
        polyline_obj = folium.PolyLine(
            locations=coordinates,
            color='blue',
            weight=2,
            opacity=0.9,
            popup=f"{name1} to {name2}"
        )
        polyline_obj.add_to(m)
        
        # Save the map as an HTML file
        map_file = f'{region}-route-map-triangle-{triangle_factor:.3f}.html'
        m.save(map_file)
        print(f"Map saved as {map_file}")

    # Display the map
    return m


region = 'ukraine'

Locations = pd.read_csv(f'{region}-locations.csv')
Distances = pd.read_csv(f'{region}-distances.csv', index_col='name')

# Apply geo_pruning function
Distances_geo = Distances.apply(geo_pruning, args=(Locations, 6, 1.1,))
Distances_geo = Distances.where(Distances_geo > 0)
Distances_geo = Distances_geo.replace(to_replace=40000000, value=np.nan)
Distances_geo['Index'] = list(Distances.columns)
Distances_geo.set_index('Index', inplace=True)
save_to_csv(Distances_geo, 'Distances_geo.csv')

# Apply triangle pruning with different factors and visualize the routes
triangle_factors = np.arange(0.8, 1.0, 0.01)
for triangle_factor in triangle_factors:
    Distances_Triangle = Distances
    Distances_Triangle = Distances_Triangle.replace(0, np.NaN)
    Distances_Triangle['Index'] = list(Distances.columns)
    Distances_Triangle.set_index('Index', inplace=True)
    Distances_Triangle = Distances_Triangle.where(np.triu(np.ones(Distances_Triangle.shape)).astype(bool))
    Distances_Triangle = trianglepruning(Distances_Triangle, triangle_factor)
    save_to_csv(Distances_Triangle, f'Distances_Triangle_{triangle_factor:.3f}.csv')

    # Create the routes DataFrame from the pruned distances
    routes_list = []
    for i, row in Distances_Triangle.iterrows():
        for col in Distances_Triangle.columns:
            distance = row[col]
            if not pd.isna(distance):
                # Ensure that 'geometry' is a valid number (not NaN) before appending to routes_list
                geometry = Distances_geo.loc[i, col]
                if not pd.isna(geometry):
                    origin = (Locations.loc[Locations['name'] == i, 'latitude'].values[0],
                              Locations.loc[Locations['name'] == i, 'longitude'].values[0])
                    destination = (Locations.loc[Locations['name'] == col, 'latitude'].values[0],
                                   Locations.loc[Locations['name'] == col, 'longitude'].values[0])
                    geometry_str = get_polyline_string(origin, destination)
                    if geometry_str is not None:
                        routes_list.append({
                            'name1': i,
                            'name2': col,
                            'geometry': geometry_str
                        })
                    else:
                        print(f"Warning: No route between {i} and {col}, skipping.")

    # Concatenate the list of dictionaries into a DataFrame
    routes_df = pd.DataFrame(routes_list)

    # Visualize the routes on the map
    m = visualize_routes_on_map(Locations, routes_df, region, triangle_factor)

    # The map will be displayed and saved for each triangle factor
    m

In [65]:
def visualize_routes_on_map(locations_df, routes_df, region, routing="knnroute"):
    # Create a map centered around the first location
    start_lat = locations_df.loc[0, 'latitude']
    start_lon = locations_df.loc[0, 'longitude']
    m_knn = folium.Map(location=[start_lat, start_lon], zoom_start=6)

    # Add markers for each location
    for i, row in locations_df.iterrows():
        name = row['name']
        lat = row['latitude']
        lon = row['longitude']
        folium.CircleMarker(
            location=[lat, lon],
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name
        ).add_to(m_knn)

    # Iterate through the routes_df DataFrame and update the geometry column
    for i, row in routes_df.iterrows():
        name1 = row['name1']
        name2 = row['name2']
        origin = (locations_df.loc[locations_df['name'] == name1, 'latitude'].values[0],
                  locations_df.loc[locations_df['name'] == name1, 'longitude'].values[0])
        destination = (locations_df.loc[locations_df['name'] == name2, 'latitude'].values[0],
                       locations_df.loc[locations_df['name'] == name2, 'longitude'].values[0])
        geometry_str = get_polyline_string(origin, destination)
        routes_df.at[i, 'geometry'] = geometry_str

        # Skip rows with NaN 'geometry' values
        if pd.isna(geometry_str):
            print(f"Warning: No route between {name1} and {name2}, skipping.")
            continue

        # Attempt to decode the polyline
        try:
            coordinates = polyline.decode(geometry_str)
        except ValueError:
            print(f"Warning: Invalid 'geometry' value for row {i}, skipping.")
            continue

        # Continue with existing code
        polyline_obj = folium.PolyLine(
            locations=coordinates,
            color='blue',
            weight=2,
            opacity=0.9,
            popup=f"{name1} to {name2}"
        )
        polyline_obj.add_to(m_knn)
        
        # Save the map as an HTML file (knnroute pruning)
        map_file_knn = f'{region}-route-map-{routing}.html'
        m_knn.save(map_file_knn)
        print(f"Map saved as {map_file_knn}")

    # Display the map
    return m

# For knnroute pruning
Distances_knn = Distances.replace(0, np.NaN)
Distances_knn['Index'] = list(Distances.columns)
Distances_knn.set_index('Index', inplace=True)
Distances_knn = Distances_knn.apply(knnroute, args=(9,))
save_to_csv(Distances_knn, 'Distances_knn.csv')
Distances_knn.head()

# Create the routes DataFrame from the pruned distances (knnroute pruning)
routes_list_knn = []
for i, row in Distances_knn.iterrows():
    for col in Distances_knn.columns:
        distance = row[col]
        if not pd.isna(distance):
            # Ensure that 'geometry' is a valid number (not NaN) before appending to routes_list_knn
            geometry = Distances_geo.loc[i, col]
            if not pd.isna(geometry):
                origin = (Locations.loc[Locations['name'] == i, 'latitude'].values[0],
                          Locations.loc[Locations['name'] == i, 'longitude'].values[0])
                destination = (Locations.loc[Locations['name'] == col, 'latitude'].values[0],
                               Locations.loc[Locations['name'] == col, 'longitude'].values[0])
                geometry_str = get_polyline_string(origin, destination)
                if geometry_str is not None:
                    routes_list_knn.append({
                        'name1': i,
                        'name2': col,
                        'geometry': geometry_str
                    })
                else:
                    print(f"Warning: No route between {i} and {col}, skipping.")

# Concatenate the list of dictionaries into a DataFrame
routes_df_knn = pd.DataFrame(routes_list_knn)

# Visualize the routes on the map (knnroute pruning)
m_knn = visualize_routes_on_map(Locations, routes_df_knn, region, "knnroute")

# Save the map as an HTML file (knnroute pruning)
map_file_knn = f'{region}-route-map-knnroute.html'
m_knn.save(map_file_knn)
print(f"Map saved as {map_file_knn}")

# Display
m_knn


Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved 

Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved as ukraine-route-map-knnroute.html
Map saved 